<a href="https://colab.research.google.com/github/devloperAnu/Sample_to_target/blob/main/updatedSTT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install ffmpeg in Google Colab
!apt-get install -y ffmpeg

# Install pydub using pip
!pip install pydub # This line installs the necessary library

# Import libraries
import os
import numpy as np
import librosa
import soundfile as sf
from google.colab import files
from pydub import AudioSegment # Now this import should work
from pydub.exceptions import CouldntDecodeError

# Function to check if ffmpeg is installed
def check_ffmpeg():
    """
    Check if ffmpeg is available in the system.

    Returns:
        bool: True if ffmpeg is found, False otherwise.
    """
    import shutil
    return shutil.which("ffmpeg") is not None

# Verify ffmpeg installation
if not check_ffmpeg():
    raise RuntimeError("ffmpeg is not installed or not found in PATH. Please install ffmpeg.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [8]:
# Upload files one by one
try:
    print("Please upload sample.mp3")
    uploaded = files.upload()
    if not uploaded:
        raise ValueError("No sample.mp3 uploaded!")
    sample_file = list(uploaded.keys())[0]

    print("Please upload target.mp3")
    uploaded = files.upload()
    if not uploaded:
        raise ValueError("No target.mp3 uploaded!")
    target_file = list(uploaded.keys())[0]

except Exception as e:
    print(f"Error during file upload: {str(e)}")
    raise

# Validate uploaded files
if not sample_file.lower().endswith('.mp3') or not target_file.lower().endswith('.mp3'):
    raise ValueError("Uploaded files must be MP3 files!")

# Define paths for conversions
output_file = "output.mp3"
sample_wav = "sample.wav"
target_wav = "target.wav"
converted_wav = "converted.wav"

Please upload sample.mp3


Saving Sample.mp3 to Sample.mp3
Please upload target.mp3


Saving target.mp3 to target.mp3


In [9]:
import os
import shutil
import subprocess
import json
from pydub import AudioSegment
from pydub.exceptions import CouldntDecodeError

# Function to check if ffmpeg and ffprobe are installed
def check_ffmpeg():
    """
    Check if ffmpeg and ffprobe are available in the system.

    Returns:
        bool: True if both ffmpeg and ffprobe are found, False otherwise.
    """
    ffmpeg_paths = ["ffmpeg", "/usr/bin/ffmpeg", "/usr/local/bin/ffmpeg"]
    ffprobe_paths = ["ffprobe", "/usr/bin/ffprobe", "/usr/local/bin/ffprobe"]
    ffmpeg_found = any(shutil.which(path) for path in ffmpeg_paths)
    ffprobe_found = any(shutil.which(path) for path in ffprobe_paths)
    return ffmpeg_found and ffprobe_found

# Function to probe audio file for diagnostics and format detection
def probe_audio(mp3_path: str) -> dict:
    """
    Run ffprobe to diagnose audio file and detect its format and codec.

    Args:
        mp3_path (str): Path to the audio file.

    Returns:
        dict: Contains format, codec, and diagnostics or error message.
    """
    if not check_ffmpeg():
        return {"error": "Cannot probe file: ffprobe not found.", "format": None, "codec": None}
    try:
        result = subprocess.run(
            ["ffprobe", "-i", mp3_path, "-show_streams", "-show_format", "-print_format", "json"],
            capture_output=True,
            text=True,
            check=False,
            timeout=10
        )
        diagnostics = result.stderr or result.stdout or "No diagnostic output available."
        if result.returncode != 0:
            return {"error": diagnostics, "format": None, "codec": None}

        # Parse ffprobe JSON output
        try:
            probe_data = json.loads(result.stdout)
            format_name = probe_data.get("format", {}).get("format_name")
            audio_stream = next((stream for stream in probe_data.get("streams", []) if stream.get("codec_type") == "audio"), None)
            codec_name = audio_stream.get("codec_name") if audio_stream else None
            return {"format": format_name, "codec": codec_name, "diagnostics": diagnostics}
        except json.JSONDecodeError:
            return {"error": "Failed to parse ffprobe output.", "format": None, "codec": None}
    except subprocess.TimeoutExpired:
        return {"error": f"Probing {mp3_path} timed out after 10 seconds.", "format": None, "codec": None}
    except Exception as e:
        return {"error": f"Failed to probe {mp3_path}: {str(e)}", "format": None, "codec": None}

# Function to convert audio file to WAV
def audio_to_wav(audio_path: str, wav_path: str) -> None:
    """
    Convert an audio file (MP3, AAC, etc.) to WAV format.

    Args:
        audio_path (str): Path to the input audio file.
        wav_path (str): Path to save the output WAV file.

    Raises:
        FileNotFoundError: If the input file or output directory does not exist.
        ValueError: If the input file is invalid.
        RuntimeError: If conversion fails due to invalid file or missing dependencies.
    """
    # Validate input file
    if not audio_path or not isinstance(audio_path, str):
        raise ValueError(f"Invalid audio path: {audio_path}")
    if not os.path.exists(audio_path):
        raise FileNotFoundError(f"Input file not found: {audio_path}")
    if os.path.getsize(audio_path) == 0:
        raise ValueError(f"Input file is empty: {audio_path}")

    # Validate output directory
    output_dir = os.path.dirname(wav_path) or '.'
    if not os.path.exists(output_dir):
        raise FileNotFoundError(f"Output directory does not exist: {output_dir}")
    if not os.access(output_dir, os.W_OK):
        raise PermissionError(f"No write permission for output directory: {output_dir}")

    # Check if output file already exists
    if os.path.exists(wav_path):
        print(f"Warning: Overwriting existing file: {wav_path}")

    # Check for ffmpeg
    if not check_ffmpeg():
        raise RuntimeError("ffmpeg or ffprobe not installed or not found in PATH. Please install ffmpeg.")

    # Probe file to detect format and codec
    probe_result = probe_audio(audio_path)
    if probe_result.get("error"):
        print(f"Probe error for {audio_path}: {probe_result['error']}")
    else:
        print(f"Detected format for {audio_path}: {probe_result['format']}, codec: {probe_result['codec']}")

    # Warn if extension doesn't match expected audio formats
    if not audio_path.lower().endswith(('.mp3', '.mpeg', '.m4a', '.aac', '.3gp', '.mp4')):
        print(f"Warning: {audio_path} has a non-standard audio extension, attempting conversion anyway.")

    try:
        # Attempt conversion with auto-detected format
        audio = AudioSegment.from_file(audio_path)
        audio.export(wav_path, format="wav")
        print(f"Successfully converted {audio_path} to {wav_path}")
    except CouldntDecodeError as e:
        print(f"Error decoding {audio_path}: {str(e)}")
        diagnostics = probe_result.get("diagnostics", "No diagnostics available.")
        print(f"Diagnostics: {diagnostics}")
        print(f"Retrying with metadata stripping and relaxed parameters...")
        try:
            # Retry with metadata stripping and relaxed parameters
            audio = AudioSegment.from_file(
                audio_path,
                parameters=["-analyzeduration", "5000000", "-probesize", "5000000", "-map_metadata", "-1", "-vn"]
            )
            audio.export(wav_path, format="wav")
            print(f"Successfully converted {audio_path} to {wav_path} on retry")
        except CouldntDecodeError as retry_e:
            print(f"Retry failed, attempting minimal processing...")
            try:
                # Fallback: minimal processing, auto-detect codec
                audio = AudioSegment.from_file(
                    audio_path,
                    parameters=["-vn", "-ignore_unknown"]
                )
                audio.export(wav_path, format="wav")
                print(f"Successfully converted {audio_path} to {wav_path} with minimal processing")
            except Exception as final_e:
                raise RuntimeError(f"Failed to decode {audio_path} after all attempts: {str(final_e)}\nDiagnostics: {diagnostics}")
        except Exception as retry_e:
            raise RuntimeError(f"Retry failed for {audio_path}: {str(retry_e)}\nDiagnostics: {diagnostics}")
    except PermissionError as e:
        raise RuntimeError(f"Permission error during conversion of {audio_path}: {str(e)}")
    except Exception as e:
        raise RuntimeError(f"Failed to convert {audio_path} to WAV: {str(e)}")

# Convert uploaded audio files to WAV
try:
    # Validate input files from Cell 2
    for var_name in ["sample_file", "target_file"]:
        try:
            var_value = globals()[var_name]
            if not var_value or not isinstance(var_value, str):
                raise ValueError(f"{var_name} is not defined or invalid. Ensure files were uploaded correctly.")
            if not os.path.exists(var_value):
                raise FileNotFoundError(f"File not found: {var_value}")
            print(f"Verified file: {var_value} (Size: {os.path.getsize(var_value)} bytes)")
        except KeyError:
            raise ValueError(f"{var_name} is not defined. Ensure Cell 2 (uploads) was executed correctly.")

    # Attempt conversions, logging errors separately
    errors = []
    try:
        audio_to_wav(sample_file, sample_wav)
    except Exception as e:
        errors.append(f"Error converting {sample_file}: {str(e)}")

    try:
        audio_to_wav(target_file, target_wav)
    except Exception as e:
        errors.append(f"Error converting {target_file}: {str(e)}")

    # Report errors if any
    if errors:
        print("Conversion errors occurred:")
        for error in errors:
            print(error)
        raise RuntimeError("One or more conversions failed. See details above.")
    else:
        print("All audio files converted successfully.")

except Exception as e:
    print(f"Error during conversion setup: {str(e)}")
    raise

Verified file: Sample.mp3 (Size: 1151462 bytes)
Verified file: target.mp3 (Size: 13834168 bytes)
Detected format for Sample.mp3: mov,mp4,m4a,3gp,3g2,mj2, codec: aac
Successfully converted Sample.mp3 to sample.wav
Detected format for target.mp3: mp3, codec: mp3
Successfully converted target.mp3 to target.wav
All audio files converted successfully.


In [12]:
import os
import numpy as np
import librosa
import pyworld
import sounddevice as sd
from scipy.io import wavfile

def analyze_and_generate_voice(file_path: str, output_path: str = None, sr: int = None, pitch_shift: float = 1.0, formant_shift: float = 1.0) -> dict:
    """
    Analyze vocal features (pitch, formants, energy) of an audio file and generate a human-like voice.

    Args:
        file_path (str): Path to the input audio file (e.g., WAV).
        output_path (str, optional): Path to save the generated audio. If None, play in real-time.
        sr (int, optional): Sampling rate. If None, use native sampling rate.
        pitch_shift (float): Factor to shift pitch (e.g., 1.2 for 20% higher, 0.8 for 20% lower).
        formant_shift (float): Factor to shift formants (controls vocal tract characteristics).

    Returns:
        dict: Dictionary containing extracted features (pitch, formants, energy) and sampling rate.

    Raises:
        FileNotFoundError: If the input file does not exist.
        ValueError: If the input file is not a WAV file.
        RuntimeError: If audio processing or synthesis fails.
    """
    # Input validation
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Audio file not found: {file_path}")
    if not file_path.lower().endswith('.wav'):
        raise ValueError(f"Input file must be a WAV file: {file_path}")

    try:
        # Load audio
        y, sr = librosa.load(file_path, sr=sr, mono=True)
        y = y.astype(np.float64)  # Ensure float64 for pyworld

        # Extract pitch (f0), formants, and energy
        f0, t = pyworld.dio(y, sr, frame_period=5.0)  # Fundamental frequency (pitch)
        sp = pyworld.cheaptrick(y, f0, t, sr)  # Spectral envelope (formants)
        ap = pyworld.d4c(y, f0, t, sr)  # Aperiodicity (voicing characteristics)
        energy = np.sqrt(np.mean(y**2))  # RMS energy

        # Clean invalid pitch values
        f0_clean = np.where(f0 > 0, f0, np.nan)
        mean_pitch = np.nanmean(f0_clean) if np.any(f0_clean) else 0
        if mean_pitch == 0:
            print(f"Warning: No valid pitch detected in {file_path}")

        # Extract formant frequencies (simplified, using spectral envelope peaks)
        formant_freqs = []
        for i in range(sp.shape[1]):
            spec = sp[:, i]
            if np.any(spec):
                peaks = np.argmax(spec[:sr//4])  # Look for peaks in lower frequencies
                formant_freqs.append(peaks * (sr / len(spec)))
        mean_formant = np.mean(formant_freqs) if formant_freqs else 0

        # Store extracted features
        features = {
            "pitch": mean_pitch,
            "formant": mean_formant,
            "energy": energy,
            "sr": sr
        }

        # Generate human-like voice by modifying pitch and formants
        # Adjust pitch (f0) and formants (spectral envelope)
        f0_mod = f0 * pitch_shift
        sp_mod = np.zeros_like(sp)
        for i in range(sp.shape[1]):
            freq_axis = np.linspace(0, sr/2, sp.shape[0])
            sp_mod[:, i] = np.interp(
                freq_axis / formant_shift, freq_axis, sp[:, i], left=0, right=0
            )

        # Synthesize new audio
        y_synth = pyworld.synthesize(f0_mod, sp_mod, ap, sr, frame_period=5.0)

        # Normalize output audio
        y_synth = y_synth / np.max(np.abs(y_synth)) * 0.9

        # Save or play the generated audio
        if output_path:
            wavfile.write(output_path, sr, y_synth.astype(np.float32))
            print(f"Generated audio saved to {output_path}")
        else:
            print("Playing generated audio...")
            sd.play(y_synth, sr)
            sd.wait()  # Wait until playback is finished

        return features

    except Exception as e:
        raise RuntimeError(f"Failed to process audio for {file_path}: {str(e)}")

# Example usage
try:
    sample_wav = "sample.wav"  # Replace with your input WAV file
    output_wav = "output_human_like.wav"  # Output file path
    features = analyze_and_generate_voice(
        sample_wav,
        output_path=output_wav,
        pitch_shift=1.1,  # Slightly higher pitch
        formant_shift=1.05  # Slightly adjusted vocal tract
    )
    print(f"Extracted features: Pitch = {features['pitch']:.2f} Hz, "
          f"Formant = {features['formant']:.2f} Hz, "
          f"Energy = {features['energy']:.4f}, "
          f"Sampling rate = {features['sr']} Hz")
except Exception as e:
    print(f"Error during voice analysis and synthesis: {str(e)}")
    raise

Generated audio saved to output_human_like.wav
Extracted features: Pitch = 130.76 Hz, Formant = 13780.06 Hz, Energy = 0.1168, Sampling rate = 44100 Hz


In [11]:
!apt-get install -qq portaudio19-dev python3-pyaudio
!pip install pyaudio
!pip install sounddevice

Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 126333 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1.1) ...
Selecting previously unselected package libportaudiocpp0:amd64.
Preparing to unpack .../libportaudiocpp0_19.6.0-1.1_amd64.deb ...
Unpacking libportaudiocpp0:amd64 (19.6.0-1.1) ...
Selecting previously unselected package portaudio19-dev:amd64.
Preparing to unpack .../portaudio19-dev_19.6.0-1.1_amd64.deb ...
Unpacking portaudio19-dev:amd64 (19.6.0-1.1) ...
Selecting previously unselected package python3-pyaudio.
Preparing to unpack .../python3-pyaudio_0.2.11-1.3ubuntu1_amd64.deb ...
Unpacking python3-pyaudio (0.2.11-1.3ubuntu1) ...
Setting up libportaudio2:amd64 (19.6.0-1.1) ...
Setting up libportaudiocpp0:amd64 (19.6.0-1.1) ...
Setting up portaudio19-dev:amd64 (19.6.0-1.1) ...
Setting up python3-pyaudio (0.2.11-1.3ubuntu1) ...
Processing triggers 

In [6]:
pip install sounddevice

In [4]:
pip install pyworld

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 8.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyworld: filename=pyworld-0.3.5-cp311-cp311-linux_x86_64.whl size=899950 sha256=7d96c2c78ed1d039a76f2e128499257b7f0e82b0c80b37be76c2376f91399ec6
  Stored in directory: /root/.cache/pip/wheels/26/f0/db/ebcd5cdfe5ad7d229917d3a8db6f18f0cf40f099bf878e294d
Successfully built pyworld


In [15]:
import os
import numpy as np
import librosa
import pyworld # Import pyworld for pitch analysis
import soundfile as sf

# Function to analyze pitch using pyworld
def analyze_pitch(file_path: str) -> dict:
    """
    Analyze pitch of an audio file using pyworld.

    Args:
        file_path (str): Path to the audio file.

    Returns:
        dict: Dictionary containing pitch and sampling rate.
    """
    try:
        y, sr = librosa.load(file_path, sr=None, mono=True)
        y = y.astype(np.float64)  # Ensure float64 for pyworld

        # Extract pitch (f0)
        f0, t = pyworld.dio(y, sr, frame_period=5.0)
        f0_clean = np.where(f0 > 0, f0, np.nan)
        mean_pitch = np.nanmean(f0_clean) if np.any(f0_clean) else 0

        return {"pitch": mean_pitch, "sr": sr}

    except Exception as e:
        raise RuntimeError(f"Failed to analyze pitch for {file_path}: {str(e)}")

def apply_pitch_shift(sample_features: dict, target_wav: str, converted_wav: str, max_semitones: float = 24) -> None:
    """
    Apply pitch shift to target audio to match the pitch of sample audio.

    Args:
        sample_features (dict): Features from sample audio (must contain 'pitch' and 'sr').
        target_wav (str): Path to the target audio file (WAV).
        converted_wav (str): Path to save the pitch-shifted audio.
        max_semitones (float): Maximum allowed pitch shift in semitones (default: 24).

    Raises:
        FileNotFoundError: If the target audio file does not exist.
        ValueError: If inputs are invalid or pitch values are unusable.
        RuntimeError: If pitch shifting or file saving fails.
    """
    # Validate inputs
    if not os.path.exists(target_wav):
        raise FileNotFoundError(f"Target audio file not found: {target_wav}")
    if not target_wav.lower().endswith('.wav'):
        raise ValueError(f"Target file must be a WAV file: {target_wav}")
    if not converted_wav.lower().endswith('.wav'):
        raise ValueError(f"Output file must be a WAV file: {converted_wav}")
    if not isinstance(sample_features, dict) or 'pitch' not in sample_features or 'sr' not in sample_features:
        raise ValueError("sample_features must be a dictionary with 'pitch' and 'sr' keys")

    # Analyze target audio pitch (assuming analyze_pitch is available)
    try:
        target_features = analyze_pitch(target_wav) # Call the defined analyze_pitch function
    except Exception as e:
        raise RuntimeError(f"Failed to analyze target audio pitch: {str(e)}")

    # Calculate pitch shift (in semitones)
    sample_pitch = sample_features['pitch']
    target_pitch = target_features['pitch']
    if target_pitch > 0 and sample_pitch > 0:
        n_steps = 12 * np.log2(sample_pitch / target_pitch)
        print(f"Pitch shift required: {n_steps:.2f} semitones")
    else:
        raise ValueError("Cannot compute pitch shift: Invalid pitch detected in sample or target audio")

    # Limit pitch shift to avoid artifacts
    if abs(n_steps) > max_semitones:
        print(f"Warning: Pitch shift ({n_steps:.2f} semitones) is too large; limiting to ±{max_semitones} semitones")
        n_steps = max(min(n_steps, max_semitones), -max_semitones)

    # Load target audio and apply pitch shift
    try:
        y_target, sr_target = librosa.load(target_wav, sr=target_features['sr'], mono=True)
        y_shifted = librosa.effects.pitch_shift(y=y_target, sr=sr_target, n_steps=n_steps)

        # Normalize output to prevent clipping
        y_shifted = y_shifted / np.max(np.abs(y_shifted)) * 0.9

        # Save the modified audio
        sf.write(converted_wav, y_shifted, sr_target)
        print(f"Pitch-shifted audio saved to {converted_wav}")

    except Exception as e:
        raise RuntimeError(f"Error during pitch shifting or saving: {str(e)}")

# Example usage
try:
    # Assume sample_features and target_features are from analyze_pitch
    sample_wav = "sample.wav"  # Replace with your sample WAV file
    target_wav = "target.wav"  # Replace with your target WAV file
    converted_wav = "converted_target.wav"  # Output file path

    # Analyze sample audio (if not already done)
    sample_features = analyze_pitch(sample_wav) # Call the defined analyze_pitch function
    print(f"Sample audio pitch: {sample_features['pitch']:.2f} Hz (Sampling rate: {sample_features['sr']} Hz)")

    # Apply pitch shift
    apply_pitch_shift(sample_features, target_wav, converted_wav)

except Exception as e:
    print(f"Error during processing: {str(e)}")
    raise

Sample audio pitch: 130.76 Hz (Sampling rate: 44100 Hz)
Pitch shift required: -5.71 semitones
Pitch-shifted audio saved to converted_target.wav


In [16]:
# Function to convert WAV to MP3
def wav_to_mp3(wav_path: str, mp3_path: str) -> None:
    """
    Convert a WAV file to MP3 format.

    Args:
        wav_path (str): Path to the input WAV file.
        mp3_path (str): Path to save the output MP3 file.

    Raises:
        FileNotFoundError: If the input WAV file or output directory does not exist.
        ValueError: If the input file is not a WAV file or output is not an MP3 file.
        RuntimeError: If conversion fails due to invalid file or missing dependencies.
    """
    if not os.path.exists(wav_path):
        raise FileNotFoundError(f"Input WAV file not found: {wav_path}")
    if not wav_path.lower().endswith('.wav'):
        raise ValueError(f"Input file must be a WAV file: {wav_path}")
    if not mp3_path.lower().endswith('.mp3'):
        raise ValueError(f"Output file must be an MP3 file: {mp3_path}")

    output_dir = os.path.dirname(mp3_path) or '.'
    if not os.path.exists(output_dir):
        raise FileNotFoundError(f"Output directory does not exist: {output_dir}")
    if not os.access(output_dir, os.W_OK):
        raise PermissionError(f"No write permission for output directory: {output_dir}")

    if os.path.exists(mp3_path):
        print(f"Warning: Overwriting existing file: {mp3_path}")

    try:
        audio = AudioSegment.from_wav(wav_path)
        audio.export(mp3_path, format="mp3")
        print(f"Successfully converted {wav_path} to {mp3_path}")
    except Exception as e:
        raise RuntimeError(f"Failed to convert {wav_path} to MP3: {str(e)}")

# Convert pitch-shifted WAV to MP3 and clean up
try:
    wav_to_mp3(converted_wav, output_file)

    # Clean up temporary WAV files
    for temp_file in [sample_wav, target_wav, converted_wav]:
        if os.path.exists(temp_file):
            try:
                os.remove(temp_file)
                print(f"Deleted temporary file: {temp_file}")
            except Exception as e:
                print(f"Warning: Failed to delete {temp_file}: {str(e)}")
        else:
            print(f"Warning: Temporary file not found: {temp_file}")

except Exception as e:
    print(f"Error during WAV-to-MP3 conversion or cleanup: {str(e)}")
    raise

Successfully converted converted_target.wav to output.mp3
Deleted temporary file: sample.wav
Deleted temporary file: target.wav
Deleted temporary file: converted_target.wav


In [17]:
# Download the output MP3
try:
    if not os.path.exists(output_file):
        raise FileNotFoundError(f"Output file not found: {output_file}")

    print(f"Downloading {output_file}...")
    files.download(output_file)
    print(f"Successfully initiated download of {output_file}")

except FileNotFoundError as e:
    print(f"Error: {str(e)}")
except Exception as e:
    print(f"Error during download: {str(e)}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Successfully initiated download of output.mp3
